# 1. Loading Libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
import numpy as np
import re
import plotly.express as px
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import SimpleRNN
from keras.models import Sequential
from keras.layers import Bidirectional
from keras.layers import Embedding
from keras.layers import GlobalAvgPool1D
from keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model

import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

import nltk
from nltk.corpus import stopwords
import string
from transformers import BertTokenizer

/kaggle/input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 2. Data Processing

In [2]:
dataset = pd.read_csv('/kaggle/input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv')
cleaned_data = dataset.dropna()
print(cleaned_data.isnull().sum())
print(cleaned_data.shape)

columns_of_interest = ['Division Name','Department Name','Class Name']
cleaned_data[columns_of_interest].nunique()

# Applying One-Hot Encoding to 'Division Name', 'Department Name', and 'Class Name'
encoded_data = pd.get_dummies(cleaned_data, columns=['Division Name', 'Department Name']).copy()

# Frequency encoding for 'Class Name'
frequency_encoding1 = encoded_data['Class Name'].value_counts(normalize=True)
frequency_encoding2 = encoded_data['Clothing ID'].value_counts(normalize=True)

# Mapping the frequencies to the original data
encoded_data['Class Name_Frequency'] = encoded_data['Class Name'].map(frequency_encoding1)
encoded_data['Clothing ID_Frequency'] = encoded_data['Clothing ID'].map(frequency_encoding2)
encoded_data.drop(columns=['Clothing ID','Class Name'],inplace = True)
encoded_data.head()

ml_data = encoded_data.copy()
ml_data = ml_data.drop(columns=['Title', 'Review Text','Unnamed: 0'])

ml_data.head()

Unnamed: 0                 0
Clothing ID                0
Age                        0
Title                      0
Review Text                0
Rating                     0
Recommended IND            0
Positive Feedback Count    0
Division Name              0
Department Name            0
Class Name                 0
dtype: int64
(19662, 11)


,Age,Rating,Recommended IND,Positive Feedback Count,Division Name_General,Division Name_General Petite,Division Name_Initmates,Department Name_Bottoms,Department Name_Dresses,Department Name_Intimate,Department Name_Jackets,Department Name_Tops,Department Name_Trend,Class Name_Frequency,Clothing ID_Frequency
2,60,3,0,0,True,False,False,False,True,False,False,False,False,0.273167,0.012766
3,50,5,1,0,False,True,False,True,False,False,False,False,False,0.058844,0.001271
4,47,5,1,6,True,False,False,False,False,False,False,True,False,0.131574,0.000203
5,49,2,0,4,True,False,False,False,True,False,False,False,False,0.273167,0.012257
6,39,5,1,1,False,True,False,False,False,False,False,True,False,0.202472,0.000915


# 3. NLP for text data

In [3]:
nlp_data = encoded_data[['Title','Review Text','Recommended IND']].copy()
nlp_data['Text'] = nlp_data['Title'] + ' ' + nlp_data['Review Text']
nlp_data = nlp_data[['Text', 'Recommended IND']]

# 1. 加载预训练的BERT模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = TFBertModel.from_pretrained('bert-base-uncased')

# 2. 去除停用词
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# 获取标点符号
punctuations = string.punctuation

def clean_text(text):
    # 分词
    tokens = tokenizer.tokenize(text)
    # 去除停用词和标点符号
    tokens = [token for token in tokens if token not in stop_words and token not in punctuations]
    # 重建文本
    return ' '.join(tokens)

# 应用清洁函数
nlp_data['Text'] = nlp_data['Text'].apply(clean_text)

# 3. 分词并统计句子长度
token_lengths = []
for text in nlp_data['Text']:
    tokens = tokenizer.encode(text, add_special_tokens=True)
    token_lengths.append(len(tokens))


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from transformers import TFBertModel, BertTokenizer
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 64

# 分词并转换为标记ID
input_ids = [tokenizer.encode(text, add_special_tokens=True) for text in nlp_data['Text']]

# 截断/填充句子
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

# 3. 注意力掩码准备
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
input_ids_ = input_ids.copy()
attention_masks_ = attention_masks.copy()

In [5]:
# 1. 数据分割
train_inputs, validation_test_inputs, train_labels, validation_test_labels = train_test_split(input_ids_, nlp_data['Recommended IND'], 
                                                                                    random_state=2018, test_size=0.3)
validation_inputs, test_inputs, validation_labels, test_labels = train_test_split(validation_test_inputs, validation_test_labels, 
                                                                                    random_state=2019, test_size=0.66)
train_masks, validation_test_masks, _, _ = train_test_split(attention_masks_, input_ids_,
                                                       random_state=2018, test_size=0.3)

validation_masks, test_masks, _, _ = train_test_split(validation_test_masks, validation_test_inputs,
                                                       random_state=2019, test_size=0.66)

# 使用相同的分割索引分割ML数据
train_ml_data, validation_test_ml_data,= train_test_split(ml_data.drop(['Recommended IND'],axis = 1).copy(), test_size=0.3, random_state=2018)

validation_ml_data, test_ml_data,= train_test_split(validation_test_ml_data, test_size=0.66, random_state=2019)

print(train_inputs.shape)

print(validation_inputs.shape)

print(test_inputs.shape)

(13763, 64)
(2005, 64)
(3894, 64)


In [6]:
# 2. 转换为Tensor
train_inputs = tf.convert_to_tensor(train_inputs)
validation_inputs = tf.convert_to_tensor(validation_inputs)
test_inputs = tf.convert_to_tensor(test_inputs)

train_labels = tf.convert_to_tensor(train_labels)
validation_labels = tf.convert_to_tensor(validation_labels)
test_labels = tf.convert_to_tensor(test_labels)

train_masks = tf.convert_to_tensor(train_masks)
validation_masks = tf.convert_to_tensor(validation_masks)
test_masks = tf.convert_to_tensor(test_masks)

train_ml_data_ml = train_ml_data.astype(float)
validation_ml_data_ml = validation_ml_data.astype(float)
test_ml_data_ml = test_ml_data.astype(float)

train_ml_data_tensor = tf.convert_to_tensor(train_ml_data_ml.values)
validation_ml_data_tensor = tf.convert_to_tensor(validation_ml_data_ml.values)
test_ml_data_tensor = tf.convert_to_tensor(test_ml_data_ml.values)

# 4. Modeling

In [7]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

def training(model, epochs):
    # 配置模型检查点
    model.summary()
    checkpoint = ModelCheckpoint(
        'model_checkpoint.h5',  # 模型保存路径和文件名
        monitor='val_loss',  # 监控验证集损失
        verbose=1,  # 输出保存信息
        save_best_only=True,  # 只保存最佳模型
        mode='min'  # 保存最小化指标的模型
    )


    # 配置早停
    early_stopping = EarlyStopping(
        monitor='val_loss',  # 监控验证集损失
        patience=10,  # 2轮无提升则停止
        restore_best_weights=True  # 恢复到最佳模型
    )

    # 训练模型
    history = model.fit(
        [train_inputs, train_masks, train_ml_data_tensor], train_labels,
        batch_size=32,
        epochs=epochs,  # 可以设置一个较大的数，因为早停会在训练过程中终止
        validation_data=([validation_inputs, validation_masks, validation_ml_data_tensor], validation_labels),
        callbacks=[early_stopping,checkpoint]  # 使用早停回调
    )
    return model

def prediction(model):
    # 对验证集进行预测
    predictions = model.predict([test_inputs, test_masks, test_ml_data_tensor])

    # 计算性能指标
    y_pred = np.round(predictions).flatten()
    y_true = test_labels.numpy()  # 如果validation_labels是Tensor

    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"Recall: {recall}\nPrecision: {precision}\nF1 Score: {f1}")


In [14]:
def RNN_Model():
    # Define input layers for text input (sequence of word indices) and attention masks
    input_ids = Input(shape=(64,), dtype=tf.int32, name='input_ids') 
    attention_masks = Input(shape=(64,), dtype=tf.int32, name='attention_masks')
    
    embedding_layer = Embedding(input_dim=len(tokenizer.vocab), output_dim=3, input_length=64)(input_ids)
    rnn_output = SimpleRNN(10)(embedding_layer)  # SimpleRNN layer
    
    rnn_output = Dense(10, activation='relu')(rnn_output)


    # Define input layer for non-text features
    ml_input = Input(shape=(14,), name='ml_input')
    
    # Apply dense layers and dropout to non-text features
    ml_output = Dense(64, activation='relu')(ml_input)
    ml_output = Dropout(0.5)(ml_output)  
    ml_output = Dense(32, activation='relu')(ml_output)  
    ml_output = Dense(16, activation='relu')(ml_output)  

    # Concatenate outputs from RNN and non-text feature branches
    combined = Concatenate()([rnn_output, ml_output])
    
    # Apply dense layers to combined features
    combined_output = Dense(64, activation='relu')(combined)
    combined_output = Dense(1, activation='sigmoid')(combined_output)

    # Build and compile the model
    model = Model(inputs=[input_ids, attention_masks, ml_input], outputs=combined_output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [9]:
def LSTM_Model():
    # Define input layers for text input (sequence of word indices) and attention masks
    input_ids = Input(shape=(64,), dtype=tf.int32, name='input_ids') 
    attention_masks = Input(shape=(64,), dtype=tf.int32, name='attention_masks')
    
    embedding_layer = Embedding(input_dim=len(tokenizer.vocab), output_dim=3, input_length=64)(input_ids)
    lstm_output = Bidirectional(LSTM(10, return_sequences=True))(embedding_layer)
    lstm_output = LSTM(10)(lstm_output)  # You can add more LSTM layers if needed
    
    # Apply dense layers and dropout to LSTM output
    lstm_output = Dense(32, activation='relu')(lstm_output)
    lstm_output = Dropout(0.4)(lstm_output)
    lstm_output = Dense(4, activation='relu')(lstm_output)

    # Define input layer for non-text features
    ml_input = Input(shape=(14,), name='ml_input')
    
    ml_output = Dense(3, activation='relu')(ml_input)  

    # Concatenate outputs from LSTM and non-text feature branches
    combined = Concatenate()([lstm_output, ml_output])
    
    # Apply dense layers to combined features
    combined_output = Dense(64, activation='relu')(combined)
    combined_output = Dense(1, activation='sigmoid')(combined_output)

    # Build and compile the model
    model = Model(inputs=[input_ids, attention_masks, ml_input], outputs=combined_output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [10]:
def GRU_Model():
    # Define input layers for text input (sequence of word indices) and attention masks
    input_ids = Input(shape=(64,), dtype=tf.int32, name='input_ids') 
    attention_masks = Input(shape=(64,), dtype=tf.int32, name='attention_masks')
    
    embedding_layer = Embedding(input_dim=len(tokenizer.vocab), output_dim=3, input_length=64)(input_ids)
    gru_output = GRU(4)(embedding_layer)  # GRU layer
    
    gru_output = Dense(1, activation='relu')(gru_output)

    # Define input layer for non-text features
    ml_input = Input(shape=(14,), name='ml_input')

    ml_output = Dense(4, activation='relu')(ml_input)  

    # Concatenate outputs from GRU and non-text feature branches
    combined = Concatenate()([gru_output, ml_output])
    
    # Apply dense layers to combined features
    combined_output = Dense(10, activation='relu')(combined)
    combined_output = Dense(1, activation='sigmoid')(combined_output)

    # Build and compile the model
    model = Model(inputs=[input_ids, attention_masks, ml_input], outputs=combined_output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [11]:
def BERT_Model():
    input_ids = Input(shape=(64,), dtype=tf.int32, name='input_ids') 
    attention_masks = Input(shape=(64,), dtype=tf.int32, name='attention_masks')
    bert_output = bert(input_ids, attention_mask=attention_masks)[1]
    bert_output = Dense(32, activation='relu')(bert_output)
    bert_output = Dropout(0.4)(bert_output)
    bert_output = Dense(32, activation='relu')(bert_output)

    for layer in bert.layers:
        layer.trainable = False

    # 4.ML数据分支
    ml_input = Input(shape=(14,), name='ml_input') # num_ml_features为ML数据的特征数量
    ml_output = Dense(64, activation='relu')(ml_input)
    ml_output = Dropout(0.5)(ml_output)  
    ml_output = Dense(32, activation='relu')(ml_output)  
    ml_output = Dense(16, activation='relu')(ml_output)  

    # 合并两个分支
    combined = Concatenate()([bert_output, ml_output])
    combined_output = Dense(64, activation='relu')(combined)
    combined_output = Dense(1, activation='sigmoid')(combined_output)

    # 构建和编译模型
    model = Model(inputs=[input_ids, attention_masks, ml_input], outputs=combined_output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model



# 5. RNN

In [25]:
rnn_model = training(RNN_Model(),2)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 ml_input (InputLayer)       [(None, 14)]                 0         []                            
                                                                                                  
 input_ids (InputLayer)      [(None, 64)]                 0         []                            
                                                                                                  
 dense_36 (Dense)            (None, 64)                   960       ['ml_input[0][0]']            
                                                                                                  
 embedding_5 (Embedding)     (None, 64, 3)                91566     ['input_ids[0][0]']           
                                                                                            

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


431/431 [==============================] - ETA: 0s - loss: 0.2817 - accuracy: 0.8797
Epoch 2: val_loss improved from 0.28836 to 0.24555, saving model to model_checkpoint.h5
431/431 [==============================] - 26s 60ms/step - loss: 0.2817 - accuracy: 0.8797 - val_loss: 0.2456 - val_accuracy: 0.9022


In [26]:
prediction(rnn_model)

122/122 [==============================] - 1s 5ms/step
Recall: 0.9041052961454089
Precision: 0.9789616559212758
F1 Score: 0.9400456174649723


# 6. LSTM

In [19]:
lstm_model = training(LSTM_Model(),2)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 64)]                 0         []                            
                                                                                                  
 embedding_3 (Embedding)     (None, 64, 3)                91566     ['input_ids[0][0]']           
                                                                                                  
 bidirectional (Bidirection  (None, 64, 20)               1120      ['embedding_3[0][0]']         
 al)                                                                                              
                                                                                                  
 lstm_1 (LSTM)               (None, 10)                   1240      ['bidirectional[0][0]'] 

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


431/431 [==============================] - ETA: 0s - loss: 0.2381 - accuracy: 0.8961
Epoch 2: val_loss improved from 0.30591 to 0.18620, saving model to model_checkpoint.h5
431/431 [==============================] - 9s 21ms/step - loss: 0.2381 - accuracy: 0.8961 - val_loss: 0.1862 - val_accuracy: 0.9192


In [20]:
prediction(lstm_model)

122/122 [==============================] - 2s 5ms/step
Recall: 0.9952992792228141
Precision: 0.8986983588002264
F1 Score: 0.9445353159851302


# 7. GRU

In [21]:
gru_model = training(GRU_Model(),2)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 64)]                 0         []                            
                                                                                                  
 embedding_4 (Embedding)     (None, 64, 3)                91566     ['input_ids[0][0]']           
                                                                                                  
 gru (GRU)                   (None, 4)                    108       ['embedding_4[0][0]']         
                                                                                                  
 ml_input (InputLayer)       [(None, 14)]                 0         []                            
                                                                                            

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


431/431 [==============================] - ETA: 0s - loss: 0.3075 - accuracy: 0.8644
Epoch 2: val_loss improved from 0.38580 to 0.23702, saving model to model_checkpoint.h5
431/431 [==============================] - 6s 15ms/step - loss: 0.3075 - accuracy: 0.8644 - val_loss: 0.2370 - val_accuracy: 0.9032


In [22]:
prediction(gru_model)

122/122 [==============================] - 1s 2ms/step
Recall: 0.9899717956753369
Precision: 0.8916172734970365
F1 Score: 0.9382239382239382


# 8. BERT

In [23]:
bert_model = training(BERT_Model(),4)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 64)]                 0         []                            
                                                                                                  
 attention_masks (InputLaye  [(None, 64)]                 0         []                            
 r)                                                                                               
                                                                                                  
 ml_input (InputLayer)       [(None, 14)]                 0         []                            
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',     

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


431/431 [==============================] - 68s 123ms/step - loss: 0.5065 - accuracy: 0.8089 - val_loss: 0.4424 - val_accuracy: 0.8185
Epoch 2/4
430/431 [============================>.] - ETA: 0s - loss: 0.3698 - accuracy: 0.8388
Epoch 2: val_loss improved from 0.44239 to 0.22735, saving model to model_checkpoint.h5
431/431 [==============================] - 49s 114ms/step - loss: 0.3698 - accuracy: 0.8388 - val_loss: 0.2274 - val_accuracy: 0.9022
Epoch 3/4
430/431 [============================>.] - ETA: 0s - loss: 0.2504 - accuracy: 0.8964
Epoch 3: val_loss improved from 0.22735 to 0.16925, saving model to model_checkpoint.h5
431/431 [==============================] - 49s 114ms/step - loss: 0.2504 - accuracy: 0.8963 - val_loss: 0.1693 - val_accuracy: 0.9272
Epoch 4/4
431/431 [==============================] - ETA: 0s - loss: 0.2063 - accuracy: 0.9154
Epoch 4: val_loss improved from 0.16925 to 0.15406, saving model to model_checkpoint.h5
431/431 [==============================] - 49s 11

In [24]:
prediction(bert_model)

122/122 [==============================] - 14s 90ms/step
Recall: 0.9486054528361015
Precision: 0.9646271510516252
F1 Score: 0.956549217885922
